# NGS Pipeline Analysis - Project 1

Comprehensive analysis of multi-condition NGS experiments including:
- RNA-seq (gene expression)
- EU-seq (nascent RNA)
- sDRIP-seq (RNA-DNA hybrids) 
- ENDseq (5' end mapping)

This notebook provides interactive analysis capabilities for the NGS pipeline with flexible treatment/control comparisons.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yaml
from pathlib import Path
import sys

# Add the project root to Python path
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Import NGS pipeline modules
from ngs_pipeline import utils, qc, peaks, quantify, viz, report
from ngs_pipeline.utils import setup_logging

# Setup
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
setup_logging()

print("✅ Libraries loaded successfully!")
print(f"📁 Project root: {project_root}")

## Configuration and Project Setup

Load project configuration and set up paths for analysis.

In [ ]:
# Load project configuration
config_file = project_root / "config" / "project_config.yml"
with open(config_file, 'r') as f:
    config = yaml.safe_load(f)

# Project information
PROJECT_NAME = config['project']['name']
DATA_DIR = Path(config['paths']['data_dir'])
RESULTS_DIR = Path(config['paths']['results_dir'])
NOTEBOOKS_DIR = Path(config['paths']['notebooks_dir'])

# Sample information
TREATMENTS = config['samples']['treatments']
CONTROLS = config['samples']['controls']
BATCHES = config['samples']['batches']
REPLICATES = config['samples']['replicates']
ASSAYS = config['assays']

print(f"🧬 Project: {PROJECT_NAME}")
print(f"📊 Treatments: {TREATMENTS}")
print(f"🔬 Controls: {CONTROLS}")
print(f"🧪 Assays: {ASSAYS}")
print(f"📋 Batches: {BATCHES}")
print(f"🔢 Replicates: {REPLICATES}")

# Display directory structure
print(f"\n📁 Directory Structure:")
print(f"   Data: {DATA_DIR}")
print(f"   Results: {RESULTS_DIR}")
print(f"   Notebooks: {NOTEBOOKS_DIR}")

## Sample Metadata Overview

Generate and explore the sample metadata for the experiment.

In [ ]:
# Generate all sample combinations
samples_data = []
for condition in TREATMENTS + CONTROLS:
    for batch in BATCHES:
        for rep in REPLICATES:
            for assay in ASSAYS:
                sample_id = f"{condition}_{batch}_rep{rep}_{assay}"
                samples_data.append({
                    'sample_id': sample_id,
                    'condition': condition,
                    'batch': batch,
                    'replicate': rep,
                    'assay': assay,
                    'treatment_type': 'treatment' if condition.startswith('Treat') else 'control',
                    'fastq_file': f"{sample_id}.fastq.gz"
                })

samples_df = pd.DataFrame(samples_data)

print(f"📊 Total samples: {len(samples_df)}")
print(f"🧬 Conditions: {samples_df['condition'].nunique()}")
print(f"🧪 Assays: {samples_df['assay'].nunique()}")
print(f"📋 Batches: {samples_df['batch'].nunique()}")

# Display sample distribution
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Samples per Condition', 'Samples per Assay', 
                   'Samples per Batch', 'Treatment vs Control'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "pie"}]]
)

# Condition distribution
condition_counts = samples_df['condition'].value_counts()
fig.add_trace(go.Bar(x=condition_counts.index, y=condition_counts.values, 
                     name='Condition'), row=1, col=1)

# Assay distribution
assay_counts = samples_df['assay'].value_counts()
fig.add_trace(go.Bar(x=assay_counts.index, y=assay_counts.values, 
                     name='Assay'), row=1, col=2)

# Batch distribution
batch_counts = samples_df['batch'].value_counts()
fig.add_trace(go.Bar(x=batch_counts.index, y=batch_counts.values, 
                     name='Batch'), row=2, col=1)

# Treatment vs Control
treatment_counts = samples_df['treatment_type'].value_counts()
fig.add_trace(go.Pie(labels=treatment_counts.index, values=treatment_counts.values, 
                     name='Type'), row=2, col=2)

fig.update_layout(height=800, title_text="Sample Distribution Overview", showlegend=False)
fig.show()

# Display first few samples
print("\n📋 Sample Overview:")
display(samples_df.head(10))

## Comparison Design Matrix

Visualize the experimental design and planned comparisons.

In [ ]:
# Create design matrix for visualization
comparisons = []

# Add primary comparisons
for comp in config['comparisons']['primary']:
    comparisons.append({
        'name': comp['name'],
        'case': comp['case'],
        'control': comp['control'],
        'type': 'primary',
        'description': comp['description']
    })

# Add batch-specific comparisons
for comp in config['comparisons']['batch_specific']:
    comparisons.append({
        'name': comp['name'],
        'case': comp['case'],
        'control': comp['control'],
        'type': 'batch_specific',
        'description': comp['description']
    })

comparisons_df = pd.DataFrame(comparisons)

print(f"🔬 Planned comparisons: {len(comparisons_df)}")
print("\n📊 Comparison Types:")
print(comparisons_df['type'].value_counts())

# Visualize comparison matrix
fig, ax = plt.subplots(figsize=(12, 8))

# Create comparison matrix
conditions = TREATMENTS + CONTROLS
comp_matrix = np.zeros((len(conditions), len(conditions)))
comp_labels = np.empty((len(conditions), len(conditions)), dtype=object)

for i, cond1 in enumerate(conditions):
    for j, cond2 in enumerate(conditions):
        if i != j:
            # Check if this comparison exists
            comp_name = f"{cond1}_vs_{cond2}"
            if any(comp_name in comp['name'] for comp in comparisons):
                comp_matrix[i, j] = 1
                comp_labels[i, j] = "✓"

# Plot heatmap
sns.heatmap(comp_matrix, 
            xticklabels=conditions, 
            yticklabels=conditions,
            annot=comp_labels, 
            fmt='', 
            cmap='RdYlBu_r',
            cbar_kws={'label': 'Comparison Planned'},
            ax=ax)

ax.set_title('Experimental Design Matrix\n(Rows = Case, Columns = Control)')
ax.set_xlabel('Control Conditions')
ax.set_ylabel('Case Conditions')
plt.tight_layout()
plt.show()

# Display comparisons table
print("\n📋 Planned Comparisons:")
display(comparisons_df)

## Data Quality Overview

Check for data availability and basic quality metrics.

In [ ]:
# Check data availability
data_status = []
missing_files = []

for _, sample in samples_df.iterrows():
    fastq_path = DATA_DIR / sample['fastq_file']
    
    if fastq_path.exists():
        # Get file size
        file_size = fastq_path.stat().st_size / (1024**2)  # MB
        data_status.append({
            'sample_id': sample['sample_id'],
            'condition': sample['condition'],
            'assay': sample['assay'],
            'file_exists': True,
            'file_size_mb': file_size
        })
    else:
        missing_files.append(sample['fastq_file'])
        data_status.append({
            'sample_id': sample['sample_id'],
            'condition': sample['condition'],
            'assay': sample['assay'],
            'file_exists': False,
            'file_size_mb': 0
        })

data_status_df = pd.DataFrame(data_status)

# Summary statistics
total_samples = len(data_status_df)
available_samples = data_status_df['file_exists'].sum()
missing_count = len(missing_files)

print(f"📊 Data Availability Summary:")
print(f"   Total samples: {total_samples}")
print(f"   Available: {available_samples}")
print(f"   Missing: {missing_count}")

if missing_count > 0:
    print(f"\n❌ Missing files:")
    for file in missing_files[:10]:  # Show first 10
        print(f"   - {file}")
    if len(missing_files) > 10:
        print(f"   ... and {len(missing_files) - 10} more")

# File size distribution by assay
if available_samples > 0:
    available_data = data_status_df[data_status_df['file_exists'] == True]
    
    fig = px.box(available_data, 
                 x='assay', 
                 y='file_size_mb',
                 color='condition',
                 title='File Size Distribution by Assay and Condition')
    fig.update_layout(height=500)
    fig.show()
    
    print(f"\n📈 File Size Statistics (MB):")
    size_stats = available_data.groupby('assay')['file_size_mb'].describe()
    display(size_stats.round(2))

## Generate Test Data

If data files are missing, generate synthetic test data for analysis.

In [ ]:
# Generate test data if needed
if missing_count > 0:
    print("🔧 Generating synthetic test data...")
    
    # Create data directory if it doesn't exist
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    
    # Import test data generator
    from test.generate_test_data import create_sample_data, create_samplesheet
    
    # Generate test data with reduced read count for speed
    create_sample_data(DATA_DIR, num_reads=1000)
    
    # Create sample sheet
    create_samplesheet(DATA_DIR)
    
    print("✅ Test data generated successfully!")
    
    # Refresh data status
    data_status = []
    for _, sample in samples_df.iterrows():
        fastq_path = DATA_DIR / sample['fastq_file']
        if fastq_path.exists():
            file_size = fastq_path.stat().st_size / (1024**2)
            data_status.append({
                'sample_id': sample['sample_id'],
                'condition': sample['condition'],
                'assay': sample['assay'],
                'file_exists': True,
                'file_size_mb': file_size
            })
    
    data_status_df = pd.DataFrame(data_status)
    print(f"📊 Updated availability: {len(data_status_df)} samples")

## RNA-seq Expression Analysis

Analyze RNA-seq data for differential expression across conditions.

In [ ]:
# Filter RNA-seq samples
rnaseq_samples = samples_df[samples_df['assay'] == 'rnaseq'].copy()
print(f"🧬 RNA-seq samples: {len(rnaseq_samples)}")

# Create mock expression data for demonstration
# In a real analysis, this would come from quantification results
np.random.seed(42)
genes = [f"GENE_{i:04d}" for i in range(1000)]
n_samples = len(rnaseq_samples)

# Generate mock count data with treatment effects
expression_data = {}
for _, sample in rnaseq_samples.iterrows():
    # Base expression + treatment effect + batch effect + noise
    base_expr = np.random.negative_binomial(20, 0.3, len(genes))
    
    # Add treatment effect for some genes
    if sample['treatment_type'] == 'treatment':
        # Upregulate some genes, downregulate others
        upregulated = np.random.choice(len(genes), 50, replace=False)
        downregulated = np.random.choice(len(genes), 30, replace=False)
        base_expr[upregulated] *= np.random.uniform(2, 5, len(upregulated))
        base_expr[downregulated] *= np.random.uniform(0.2, 0.8, len(downregulated))
    
    # Add batch effect
    if sample['batch'] == 'batch2':
        base_expr *= np.random.uniform(0.8, 1.2, len(genes))
    
    expression_data[sample['sample_id']] = base_expr

# Create expression matrix
expr_df = pd.DataFrame(expression_data, index=genes)
print(f"📊 Expression matrix: {expr_df.shape[0]} genes × {expr_df.shape[1]} samples")

# Normalize (log2 + 1 transformation)
expr_log = np.log2(expr_df + 1)

# PCA analysis
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Standardize data
scaler = StandardScaler()
expr_scaled = scaler.fit_transform(expr_log.T)

# PCA
pca = PCA(n_components=3)
pca_result = pca.fit_transform(expr_scaled)

# Add PCA results to sample info
rnaseq_samples['PC1'] = pca_result[:, 0]
rnaseq_samples['PC2'] = pca_result[:, 1]
rnaseq_samples['PC3'] = pca_result[:, 2]

# Plot PCA
fig = px.scatter(rnaseq_samples, 
                 x='PC1', y='PC2',
                 color='condition',
                 symbol='batch',
                 title=f'PCA of RNA-seq Samples<br>PC1: {pca.explained_variance_ratio_[0]:.1%}, PC2: {pca.explained_variance_ratio_[1]:.1%}',
                 hover_data=['sample_id'])
fig.update_layout(height=500)
fig.show()

print(f"📈 PCA Explained Variance:")
for i, var in enumerate(pca.explained_variance_ratio_[:3]):
    print(f"   PC{i+1}: {var:.1%}")

## Differential Expression Analysis

Perform differential expression analysis for key comparisons.

In [ ]:
# Mock differential expression results
# In practice, this would use DESeq2 or similar tools

def mock_differential_expression(case_condition, control_condition, expr_df, sample_info):
    """Generate mock differential expression results."""
    
    # Get samples for comparison
    case_samples = sample_info[sample_info['condition'] == case_condition]['sample_id'].tolist()
    control_samples = sample_info[sample_info['condition'] == control_condition]['sample_id'].tolist()
    
    if not case_samples or not control_samples:
        print(f"⚠️ No samples found for {case_condition} vs {control_condition}")
        return None
    
    # Calculate mean expression
    case_mean = expr_df[case_samples].mean(axis=1)
    control_mean = expr_df[control_samples].mean(axis=1)
    
    # Calculate fold change
    log2fc = case_mean - control_mean
    
    # Mock p-values (random but consistent)
    np.random.seed(hash(case_condition + control_condition) % 2**32)
    pvalues = np.random.beta(0.5, 10, len(expr_df))  # Most genes not significant
    
    # Adjust some p-values for genes with large fold changes
    large_fc_genes = np.abs(log2fc) > 1
    pvalues[large_fc_genes] *= 0.1  # Make them more significant
    
    # FDR correction (simplified)
    from scipy import stats
    _, padj = stats.false_discovery_control(pvalues, method='benjamini-hochberg')
    
    # Create results dataframe
    results = pd.DataFrame({
        'gene_id': expr_df.index,
        'case_mean': case_mean,
        'control_mean': control_mean,
        'log2FoldChange': log2fc,
        'pvalue': pvalues,
        'padj': padj
    })
    
    # Add significance flags
    results['significant'] = (results['padj'] < 0.05) & (np.abs(results['log2FoldChange']) > 1)
    results['direction'] = np.where(results['log2FoldChange'] > 0, 'Up', 'Down')
    
    return results.sort_values('pvalue')

# Run differential expression for primary comparisons
de_results = {}
for comp in config['comparisons']['primary']:
    case = comp['case']
    control = comp['control']
    comp_name = comp['name']
    
    print(f"🔬 Running DE analysis: {comp_name}")
    results = mock_differential_expression(case, control, expr_log, rnaseq_samples)
    
    if results is not None:
        de_results[comp_name] = results
        
        n_total = len(results)
        n_sig = results['significant'].sum()
        n_up = ((results['significant']) & (results['log2FoldChange'] > 0)).sum()
        n_down = ((results['significant']) & (results['log2FoldChange'] < 0)).sum()
        
        print(f"   Total genes: {n_total}")
        print(f"   Significant: {n_sig} ({n_sig/n_total:.1%})")
        print(f"   Upregulated: {n_up}")
        print(f"   Downregulated: {n_down}")

print(f"\n✅ Completed {len(de_results)} differential expression analyses")

## Visualization Dashboard

Create interactive plots for exploring differential expression results.

In [ ]:
# Create multi-panel visualization dashboard
from plotly.subplots import make_subplots

def create_de_dashboard(de_results):
    """Create interactive dashboard for DE results."""
    
    n_comparisons = len(de_results)
    if n_comparisons == 0:
        print("No DE results to display")
        return
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('DE Summary', 'Volcano Plot', 'MA Plot', 'Top Genes'),
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "scatter"}, {"type": "bar"}]]
    )
    
    # Summary statistics
    summary_data = []
    for comp_name, results in de_results.items():
        n_up = ((results['significant']) & (results['log2FoldChange'] > 0)).sum()
        n_down = ((results['significant']) & (results['log2FoldChange'] < 0)).sum()
        summary_data.extend([
            {'comparison': comp_name, 'direction': 'Up', 'count': n_up},
            {'comparison': comp_name, 'direction': 'Down', 'count': n_down}
        ])
    
    summary_df = pd.DataFrame(summary_data)
    
    # Plot summary
    for direction in ['Up', 'Down']:
        data = summary_df[summary_df['direction'] == direction]
        fig.add_trace(go.Bar(
            x=data['comparison'], 
            y=data['count'],
            name=f'{direction}regulated',
            marker_color='red' if direction == 'Up' else 'blue'
        ), row=1, col=1)
    
    # Volcano plot (first comparison)
    if de_results:
        first_comp = list(de_results.keys())[0]
        first_results = de_results[first_comp]
        
        # Volcano plot
        colors = ['red' if sig and lfc > 0 else 'blue' if sig and lfc < 0 else 'gray' 
                 for sig, lfc in zip(first_results['significant'], first_results['log2FoldChange'])]
        
        fig.add_trace(go.Scatter(
            x=first_results['log2FoldChange'],
            y=-np.log10(first_results['pvalue']),
            mode='markers',
            marker=dict(color=colors, size=4, opacity=0.6),
            text=first_results['gene_id'],
            name='Genes',
            showlegend=False
        ), row=1, col=2)
        
        # MA plot
        avg_expr = (first_results['case_mean'] + first_results['control_mean']) / 2
        fig.add_trace(go.Scatter(
            x=avg_expr,
            y=first_results['log2FoldChange'],
            mode='markers',
            marker=dict(color=colors, size=4, opacity=0.6),
            text=first_results['gene_id'],
            name='Genes',
            showlegend=False
        ), row=2, col=1)
        
        # Top genes
        top_genes = first_results.nsmallest(20, 'padj')
        fig.add_trace(go.Bar(
            x=-np.log10(top_genes['padj']),
            y=top_genes['gene_id'],
            orientation='h',
            name='Top Genes',
            showlegend=False
        ), row=2, col=2)
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text=f"Differential Expression Dashboard - {PROJECT_NAME}",
        showlegend=True
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Comparison", row=1, col=1)
    fig.update_yaxes(title_text="Number of Genes", row=1, col=1)
    
    fig.update_xaxes(title_text="log2 Fold Change", row=1, col=2)
    fig.update_yaxes(title_text="-log10(p-value)", row=1, col=2)
    
    fig.update_xaxes(title_text="Average Expression", row=2, col=1)
    fig.update_yaxes(title_text="log2 Fold Change", row=2, col=1)
    
    fig.update_xaxes(title_text="-log10(adj p-value)", row=2, col=2)
    fig.update_yaxes(title_text="Gene", row=2, col=2)
    
    return fig

# Create and display dashboard
dashboard = create_de_dashboard(de_results)
if dashboard:
    dashboard.show()

# Display results summary
if de_results:
    print("\n📊 Differential Expression Summary:")
    for comp_name, results in de_results.items():
        sig_genes = results[results['significant']]
        print(f"\n🔬 {comp_name}:")
        print(f"   Significant genes: {len(sig_genes)}")
        if len(sig_genes) > 0:
            print(f"   Top upregulated: {sig_genes[sig_genes['log2FoldChange'] > 0].iloc[0]['gene_id'] if len(sig_genes[sig_genes['log2FoldChange'] > 0]) > 0 else 'None'}")
            print(f"   Top downregulated: {sig_genes[sig_genes['log2FoldChange'] < 0].iloc[0]['gene_id'] if len(sig_genes[sig_genes['log2FoldChange'] < 0]) > 0 else 'None'}")

## Export Results

Save analysis results and generate reports.

In [ ]:
# Create results directory
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Save differential expression results
de_output_dir = RESULTS_DIR / "differential_expression"
de_output_dir.mkdir(exist_ok=True)

for comp_name, results in de_results.items():
    output_file = de_output_dir / f"{comp_name}_results.tsv"
    results.to_csv(output_file, sep='\t', index=False)
    print(f"💾 Saved: {output_file}")

# Save sample metadata
samples_output = RESULTS_DIR / "sample_metadata.tsv"
samples_df.to_csv(samples_output, sep='\t', index=False)
print(f"💾 Saved: {samples_output}")

# Save expression matrix
expr_output = RESULTS_DIR / "expression_matrix.tsv"
expr_df.to_csv(expr_output, sep='\t')
print(f"💾 Saved: {expr_output}")

# Generate summary report
summary_report = {
    'project': PROJECT_NAME,
    'date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_samples': len(samples_df),
    'conditions': samples_df['condition'].unique().tolist(),
    'assays': ASSAYS,
    'comparisons_analyzed': list(de_results.keys()),
    'total_genes': len(expr_df) if len(expr_df) > 0 else 0,
    'significant_genes_per_comparison': {
        comp: int(results['significant'].sum()) 
        for comp, results in de_results.items()
    }
}

# Save summary
summary_output = RESULTS_DIR / "analysis_summary.json"
import json
with open(summary_output, 'w') as f:
    json.dump(summary_report, f, indent=2)

print(f"💾 Saved: {summary_output}")

print(f"\n✅ Analysis complete!")
print(f"📁 Results saved to: {RESULTS_DIR}")
print(f"🔬 Analyzed {len(de_results)} comparisons")
print(f"🧬 Processed {len(samples_df)} samples across {len(ASSAYS)} assays")

## Next Steps

This notebook provides a foundation for NGS analysis with the new pipeline structure. Key features:

### ✅ Completed in this Analysis:
- Project-based organization with flexible folder structure
- Multi-condition experimental design (Treat1/Treat2 vs Control1/Control2)
- Batch and replicate handling
- Interactive visualization dashboard
- Configurable comparison system

### 🔄 For Production Analysis:
1. **Real Data Processing**: Replace mock data with actual sequencing results
2. **Snakemake Workflow**: Use `workflow/Snakefile` for automated processing
3. **Quality Control**: Run comprehensive QC with FastQC/MultiQC
4. **Peak Analysis**: Analyze sDRIP-seq and ENDseq peaks
5. **Integration**: Combine results across assay types

### 🛠️ Pipeline Commands:
```bash
# Run Snakemake workflow
snakemake -s workflow/Snakefile --configfile config/project_config.yml --cores 8

# Run individual analysis modules
ngs_pipeline qc --input data/project1/ --output results/project1/qc/
ngs_pipeline quantify --input results/project1/mapping/ --output results/project1/expression/
```

### 📊 Configuration:
- Modify `config/project_config.yml` for different comparisons
- Add new conditions, batches, or assays as needed
- Customize analysis parameters

The pipeline now supports flexible experimental designs with batch effects, multiple treatments/controls, and comprehensive visualization capabilities.